In [1]:
import pandas as pd
import query_elasticsearch as es

# Claim 
re_step1_results = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_restep1results.csv"
claim = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_claim.csv"
perspective = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_perspective.csv"

re_step1_df = pd.read_csv(re_step1_results)
claim_df = pd.read_csv(claim)
perspective_df = pd.read_csv(perspective)
perspective_df = perspective_df.dropna()

re_step1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27507 entries, 0 to 27506
Data columns (total 10 columns):
id                        27507 non-null int64
claim_id                  27507 non-null int64
perspective_id            27507 non-null int64
vote_support              27507 non-null int64
vote_leaning_support      27507 non-null int64
vote_leaning_undermine    27507 non-null int64
vote_undermine            27507 non-null int64
vote_not_valid            27507 non-null int64
p_i_5                     27507 non-null float64
p_i_3                     27507 non-null float64
dtypes: float64(2), int64(8)
memory usage: 2.1 MB


In [2]:
perspective_df.info()
claim_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20797 entries, 0 to 20797
Data columns (total 5 columns):
id                       20797 non-null int64
source                   20797 non-null object
title                    20797 non-null object
pilot1_high_agreement    20797 non-null int64
similar_persps           20797 non-null object
dtypes: int64(2), object(3)
memory usage: 974.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 7 columns):
id                          1006 non-null int64
source                      1006 non-null object
title                       1006 non-null object
assignment_counts           1006 non-null int64
finished_counts             1006 non-null int64
evidence_assign_counts      1006 non-null int64
evidence_finished_counts    1006 non-null int64
dtypes: int64(5), object(2)
memory usage: 55.1+ KB


In [3]:
# Filter out low quality ones
th = 0.5
re_step1_df = re_step1_df[re_step1_df["p_i_3"] >= th]
re_step1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13560 entries, 0 to 23877
Data columns (total 10 columns):
id                        13560 non-null int64
claim_id                  13560 non-null int64
perspective_id            13560 non-null int64
vote_support              13560 non-null int64
vote_leaning_support      13560 non-null int64
vote_leaning_undermine    13560 non-null int64
vote_undermine            13560 non-null int64
vote_not_valid            13560 non-null int64
p_i_5                     13560 non-null float64
p_i_3                     13560 non-null float64
dtypes: float64(2), int64(8)
memory usage: 1.1 MB


In [4]:
# For each claim + perspective pair in results, we index them in lucene (elastic search)
def get_claim(claim_id):
    q = claim_df[claim_df.id == claim_id]
    if len(q) > 0:
        return q.iloc[0].title
    else:
        print("Claim not found! cid = {}".format(claim_id))
        return None

def get_perspective(perspective_id):
    q = perspective_df[perspective_df.id == perspective_id]
    if len(q) > 0:
        return q.iloc[0].title
    else:
        print("Perspective not found! pid = {}".format(perspective_id))
        return None

def concat_claim_persp(claim, persp):
    if not claim.endswith("."):
        try:
            return claim + ". " + persp
        except TypeError:
            print(claim, persp)
    else:
        return claim + " " + persp

data = []
for idx, row in re_step1_df.iterrows():
    cid = row.claim_id
    pid = row.perspective_id
    claim_title = get_claim(cid)
    persp_title = get_perspective(pid)
    
    if persp_title and claim_title:
        c_p_concat = concat_claim_persp(claim_title, persp_title)
        data.append({
            "claim_id" : int(cid), 
            "perspective_id" : int(pid), 
            "concat_title" : c_p_concat
        })

# save data, for later usage
import json

out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot12_evidence_verification/claim_persp.json"
with open(out_path, 'w') as fout:
    json.dump(data, fout)

Perspective not found! pid = 5036.0


In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://bronte.cs.illinois.edu'],port=8080)

# Create indices 
index_name = "re_step1_claim_persp_high_quality"

# es.indices.delete(index_name)
es.indices.create(index_name)
    
for doc in data:
    cpid = str(doc["claim_id"]) + "_" + str(doc["perspective_id"])
    es.index(index=index_name, doc_type='text', id=cpid, body=doc)



In [8]:
from elasticsearch import TransportError
# For each evidence, retrieve the list of perspectives 
def get_top_re_step1_perspectives(text, num_cands=20):
    res = es.search(index="re_step1_claim_persp_high_quality", doc_type="text", body={"query": {"match": {"concat_title": text}}}, size=num_cands)
    # print("%d documents found:" % res['hits']['total'])
    output = []
    for doc in res['hits']['hits']:
        cid = doc['_source']["claim_id"]
        pid = doc['_source']["perspective_id"]
        score = doc['_score']
        perspective_text = doc['_source']["concat_title"]
        output.append((perspective_text, cid, pid, score))

    return output

evidence_path = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_evidence.csv"

edf = pd.read_csv(evidence_path)

result = []
for idx, row in edf.iterrows():
    print("Querying evidence: {}".format(row.id))
    try:
        output = get_top_re_step1_perspectives(row.content)
    except TransportError:
        output = []
        
    result.append({
        "evidence_id": row.id,
        "evidence_text": row.content,
        "candidates": output
    })

out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot12_evidence_verification/perspective_candidates.json"
with open(out_path, 'w') as fout:
    json.dump(result, fout)

Querying evidence: 1
Querying evidence: 2
Querying evidence: 3
Querying evidence: 4
Querying evidence: 5
Querying evidence: 6
Querying evidence: 7
Querying evidence: 8
Querying evidence: 9
Querying evidence: 10
Querying evidence: 11
Querying evidence: 12
Querying evidence: 13
Querying evidence: 14
Querying evidence: 15
Querying evidence: 16
Querying evidence: 17
Querying evidence: 18
Querying evidence: 19
Querying evidence: 20
Querying evidence: 21
Querying evidence: 22
Querying evidence: 23
Querying evidence: 24
Querying evidence: 25
Querying evidence: 26
Querying evidence: 27
Querying evidence: 28
Querying evidence: 29
Querying evidence: 30
Querying evidence: 31
Querying evidence: 32
Querying evidence: 33
Querying evidence: 34
Querying evidence: 35
Querying evidence: 36
Querying evidence: 37
Querying evidence: 38
Querying evidence: 39
Querying evidence: 40
Querying evidence: 41
Querying evidence: 42
Querying evidence: 43
Querying evidence: 44
Querying evidence: 45
Querying evidence: 

Querying evidence: 366
Querying evidence: 367
Querying evidence: 368
Querying evidence: 369
Querying evidence: 370
Querying evidence: 371
Querying evidence: 372
Querying evidence: 373
Querying evidence: 374
Querying evidence: 375
Querying evidence: 376
Querying evidence: 377
Querying evidence: 378
Querying evidence: 379
Querying evidence: 380
Querying evidence: 381
Querying evidence: 382
Querying evidence: 383
Querying evidence: 384
Querying evidence: 385
Querying evidence: 386
Querying evidence: 387
Querying evidence: 388
Querying evidence: 389
Querying evidence: 390
Querying evidence: 391
Querying evidence: 392
Querying evidence: 393
Querying evidence: 394
Querying evidence: 395
Querying evidence: 396
Querying evidence: 397
Querying evidence: 398
Querying evidence: 399
Querying evidence: 400
Querying evidence: 401
Querying evidence: 402
Querying evidence: 403
Querying evidence: 404
Querying evidence: 405
Querying evidence: 406
Querying evidence: 407
Querying evidence: 408
Querying ev

Querying evidence: 726
Querying evidence: 727
Querying evidence: 728
Querying evidence: 729
Querying evidence: 730
Querying evidence: 731
Querying evidence: 732
Querying evidence: 733
Querying evidence: 734
Querying evidence: 735
Querying evidence: 736
Querying evidence: 737
Querying evidence: 738
Querying evidence: 739
Querying evidence: 740
Querying evidence: 741
Querying evidence: 742
Querying evidence: 743
Querying evidence: 744
Querying evidence: 745
Querying evidence: 746
Querying evidence: 747
Querying evidence: 748
Querying evidence: 749
Querying evidence: 750
Querying evidence: 751
Querying evidence: 752
Querying evidence: 753
Querying evidence: 754
Querying evidence: 755
Querying evidence: 756
Querying evidence: 757
Querying evidence: 758
Querying evidence: 759
Querying evidence: 760
Querying evidence: 761
Querying evidence: 762
Querying evidence: 763
Querying evidence: 764
Querying evidence: 765
Querying evidence: 766
Querying evidence: 767
Querying evidence: 768
Querying ev

Querying evidence: 1081
Querying evidence: 1082
Querying evidence: 1083
Querying evidence: 1084
Querying evidence: 1085
Querying evidence: 1086
Querying evidence: 1087
Querying evidence: 1088
Querying evidence: 1089
Querying evidence: 1090
Querying evidence: 1091
Querying evidence: 1092
Querying evidence: 1093
Querying evidence: 1094
Querying evidence: 1095
Querying evidence: 1096
Querying evidence: 1097
Querying evidence: 1098
Querying evidence: 1099
Querying evidence: 1100
Querying evidence: 1101
Querying evidence: 1102
Querying evidence: 1103
Querying evidence: 1104
Querying evidence: 1105
Querying evidence: 1106
Querying evidence: 1107
Querying evidence: 1108
Querying evidence: 1109
Querying evidence: 1110
Querying evidence: 1111
Querying evidence: 1112
Querying evidence: 1113
Querying evidence: 1114
Querying evidence: 1115
Querying evidence: 1116
Querying evidence: 1117
Querying evidence: 1118
Querying evidence: 1119
Querying evidence: 1120
Querying evidence: 1121
Querying evidenc

Querying evidence: 1424
Querying evidence: 1425
Querying evidence: 1426
Querying evidence: 1427
Querying evidence: 1428
Querying evidence: 1429
Querying evidence: 1430
Querying evidence: 1431
Querying evidence: 1432
Querying evidence: 1433
Querying evidence: 1434
Querying evidence: 1435
Querying evidence: 1436
Querying evidence: 1437
Querying evidence: 1438
Querying evidence: 1439
Querying evidence: 1440
Querying evidence: 1441
Querying evidence: 1442
Querying evidence: 1443
Querying evidence: 1444
Querying evidence: 1445
Querying evidence: 1446
Querying evidence: 1447
Querying evidence: 1448
Querying evidence: 1449
Querying evidence: 1450
Querying evidence: 1451
Querying evidence: 1452
Querying evidence: 1453
Querying evidence: 1454
Querying evidence: 1455
Querying evidence: 1456
Querying evidence: 1457
Querying evidence: 1458
Querying evidence: 1459
Querying evidence: 1460
Querying evidence: 1461
Querying evidence: 1462
Querying evidence: 1463
Querying evidence: 1464
Querying evidenc

...

Querying evidence: 7994
Querying evidence: 7995
Querying evidence: 7996
Querying evidence: 7997
Querying evidence: 7998
Querying evidence: 7999
Querying evidence: 8000
Querying evidence: 8001
Querying evidence: 8002
Querying evidence: 8003
Querying evidence: 8004
Querying evidence: 8005
Querying evidence: 8006
Querying evidence: 8007
Querying evidence: 8008
Querying evidence: 8009
Querying evidence: 8010
Querying evidence: 8011
Querying evidence: 8012
Querying evidence: 8013
Querying evidence: 8014
Querying evidence: 8015
Querying evidence: 8016
Querying evidence: 8017
Querying evidence: 8018
Querying evidence: 8019
Querying evidence: 8020
Querying evidence: 8021
Querying evidence: 8022
Querying evidence: 8023
Querying evidence: 8024
Querying evidence: 8025
Querying evidence: 8026
Querying evidence: 8027
Querying evidence: 8028
Querying evidence: 8029
Querying evidence: 8030
Querying evidence: 8031
Querying evidence: 8032
Querying evidence: 8033
Querying evidence: 8034
Querying evidenc

GET http://bronte.cs.illinois.edu:8080/re_step1_claim_persp_high_quality/text/_search?size=20 [status:400 request:0.070s]
GET http://bronte.cs.illinois.edu:8080/re_step1_claim_persp_high_quality/text/_search?size=20 [status:400 request:0.069s]


Querying evidence: 8067
Querying evidence: 8068
Querying evidence: 8069
Querying evidence: 8070
Querying evidence: 8071
Querying evidence: 8072
Querying evidence: 8073
Querying evidence: 8074
Querying evidence: 8075
Querying evidence: 8076
Querying evidence: 8077
Querying evidence: 8078
Querying evidence: 8079
Querying evidence: 8080
Querying evidence: 8081
Querying evidence: 8082
Querying evidence: 8083
Querying evidence: 8084
Querying evidence: 8085
Querying evidence: 8086
Querying evidence: 8087
Querying evidence: 8088
Querying evidence: 8089
Querying evidence: 8090


GET http://bronte.cs.illinois.edu:8080/re_step1_claim_persp_high_quality/text/_search?size=20 [status:400 request:0.097s]


Querying evidence: 8091
Querying evidence: 8092
Querying evidence: 8093
Querying evidence: 8094
Querying evidence: 8095
Querying evidence: 8096
Querying evidence: 8097
Querying evidence: 8098
Querying evidence: 8099
Querying evidence: 8100
Querying evidence: 8101
Querying evidence: 8102
Querying evidence: 8103
Querying evidence: 8104
Querying evidence: 8105
Querying evidence: 8106
Querying evidence: 8107
Querying evidence: 8108
Querying evidence: 8109
Querying evidence: 8110
Querying evidence: 8111
Querying evidence: 8112
Querying evidence: 8113
Querying evidence: 8114
Querying evidence: 8115
Querying evidence: 8116
Querying evidence: 8117
Querying evidence: 8118
Querying evidence: 8119
Querying evidence: 8120
Querying evidence: 8121
Querying evidence: 8122


In [16]:
# Index evidences
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://bronte.cs.illinois.edu'],port=8080, timeout=60)

# Create indices 
index_name = "evidences"

# es.indices.delete(index_name)
es.indices.create(index_name)

evidence_path = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_evidence.csv"
edf = pd.read_csv(evidence_path)
edf = edf.dropna()
edf.info()

result = []
for idx, row in edf.iterrows():
    if row.content:
        doc = {
            "id" : row.id,
            "content" : row.content
        }
        es.index(index=index_name, doc_type='text', id=row.id, body=doc)    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8092 entries, 0 to 8121
Data columns (total 3 columns):
id         8092 non-null int64
source     8092 non-null object
content    8092 non-null object
dtypes: int64(1), object(2)
memory usage: 252.9+ KB


In [27]:
es = Elasticsearch(['http://bronte.cs.illinois.edu'],port=8080, timeout=30)
cp_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot12_evidence_verification/claim_persp.json"

def get_top_evidences(text, num_cands=50):
    res = es.search(index="evidences", doc_type="text", body={"query": {"match": {"content": text}}}, size=num_cands)
    # print("%d documents found:" % res['hits']['total'])
    output = []
    for doc in res['hits']['hits']:
        eid = doc['_source']["id"]
        score = doc['_score']
        evidence = doc['_source']["content"]
        output.append((evidence, eid, score))

    return output

with open(cp_path) as fin:
    cp_data = json.load(fin)

# Evidence 
# For each claim + perspctive, find evidence candidates, 
persp_candidates = {}

for cp in cp_data:
    cid = cp["claim_id"]
    pid = cp["perspective_id"]
    
    for evidence, eid, score in get_top_evidences(cp["concat_title"]):
        if eid not in persp_candidates:
            persp_candidates[eid] = []
        
        token_len = len(cp["concat_title"].split(" "))
        persp_candidates[eid].append((cid, pid, score/token_len))



In [28]:
# Sort 
for eid in persp_candidates:
    persp_candidates[eid] = sorted(persp_candidates[eid], key=lambda trip: trip[2], reverse=True)

out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot12_evidence_verification/reverse_persp_candidates.json"

with open(out_path, 'w') as fout:
    json.dump(persp_candidates, fout)

In [29]:
from collections import OrderedDict


er = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_evidencerelation.csv"

er_df = pd.read_csv(er)

k = [1, 5, 20, 50]
correct = {_k: 0 for _k in k}

for eid in persp_candidates:
    gold_pid = er_df[er_df.evidence_id == eid].iloc[0].perspective_id
        
    p_cands = [t[1] for t in persp_candidates[eid]]
    p_cands = list(OrderedDict.fromkeys(p_cands))
    
    for _k in k:
        _cands = p_cands[:_k]
        if gold_pid in _cands:
            correct[_k] += 1

total = len(persp_candidates)

for _k in correct:
    print("Top {} accuracy: {}".format(_k, correct[_k]/total))

Top 1 accuracy: 0.15717821782178218
Top 5 accuracy: 0.3311881188118812
Top 20 accuracy: 0.4498762376237624
Top 50 accuracy: 0.48824257425742573


In [26]:
in_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot12_evidence_verification/perspective_candidates.json"

k = [1, 5, 20, 50]
correct = {_k: 0 for _k in k}

with open(in_path) as fin:
    ev2pp = json.load(fin)
    for ev in ev2pp:
        eid = ev["evidence_id"]
        gold_pid = er_df[er_df.evidence_id == eid].iloc[0].perspective_id
        cands = [p[2] for p in ev["candidates"]]
        
        for _k in k:
            _cands = cands[:_k]
            if gold_pid in _cands:
                correct[_k] += 1
            
total = len(ev2pp)

for _k in correct:
    print("Top {} accuracy: {}".format(_k, correct[_k]/total))

Top 1 accuracy: 0.1740950504801773
Top 5 accuracy: 0.2890913568086678
Top 20 accuracy: 0.3683821718788476
Top 50 accuracy: 0.3683821718788476
